## Identifies all tribtaries within the DEM that are >1km^2

In [16]:
import numpy as np
import rasterio
from rasterio.transform import xy
import pandas as pd
from scipy import ndimage

# ============== PARAMETERS TO ADJUST ==============
BASE_PATH = "/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/"
DEM_FILE = BASE_PATH + "rapel_SRTMGL130m_dem_utm.tif"
AREA_FILE = BASE_PATH + "rapel_area_utm30m"
FD_FILE = BASE_PATH + "rapel_fd_utm30m"

# Drainage area thresholds (in square meters)
MIN_DRAINAGE_AREA = 5_000_000  # 1 km²
MAX_DRAINAGE_AREA = 500_000_000  #  km² - adjust as needed

# Elevation threshold (in meters)
MAX_ELEVATION = 1000

# Spatial extent limits (UTM X coordinates)
MIN_X = 166836
MAX_X = 341552

# Ocean boundary detection
CONSECUTIVE_OCEAN_LIMIT = 2  # Number of consecutive ocean cells (elev ≤ 0) to indicate ocean boundary

# Limit number of outlets for testing (set to None for all)
MAX_OUTLETS = None

# Output file
OUTPUT_CSV = BASE_PATH + "tributary_outlets.csv"

# D8 flow direction encoding (ESRI/TauDEM style)
# 1=E, 2=SE, 4=S, 8=SW, 16=W, 32=NW, 64=N, 128=NE
D8_DIRECTIONS = {
    1: (0, 1),    # East
    2: (1, 1),    # Southeast
    4: (1, 0),    # South
    8: (1, -1),   # Southwest
    16: (0, -1),  # West
    32: (-1, -1), # Northwest
    64: (-1, 0),  # North
    128: (-1, 1)  # Northeast
}

# ============== FUNCTIONS ==============

def load_raster(filepath):
    """Load a raster file and return data array and metadata."""
    with rasterio.open(filepath) as src:
        data = src.read(1)
        profile = src.profile
        transform = src.transform
        crs = src.crs
    return data, profile, transform, crs

def get_downstream_cell(row, col, fd_value):
    """Get the downstream cell coordinates based on D8 flow direction."""
    if fd_value in D8_DIRECTIONS:
        dr, dc = D8_DIRECTIONS[fd_value]
        return row + dr, col + dc
    return None, None

def get_upstream_cells(row, col, fd):
    """
    Find all cells that flow into the current cell.
    Returns list of (row, col) tuples.
    """
    upstream = []
    
    # Check all 8 neighbors
    for fd_val, (dr, dc) in D8_DIRECTIONS.items():
        neighbor_row = row - dr  # Reverse direction to find upstream
        neighbor_col = col - dc
        
        # Check if neighbor is in bounds
        if (0 <= neighbor_row < fd.shape[0] and 
            0 <= neighbor_col < fd.shape[1]):
            
            # Check if neighbor flows into current cell
            if fd[neighbor_row, neighbor_col] == fd_val:
                upstream.append((neighbor_row, neighbor_col))
    
    return upstream

def check_upstream_path_for_ocean(row, col, elevation, fd, consecutive_limit=3, max_steps=10):
    """
    Trace upstream from the current cell and check if we encounter multiple
    consecutive ocean cells (elevation ≤ 0), which would indicate we're in the ocean
    rather than at a true river outlet.
    
    Returns:
    --------
    bool : True if path goes through ocean (should reject), False if valid outlet
    """
    consecutive_ocean = 0
    visited = set()
    current_cells = [(row, col)]
    
    for step in range(max_steps):
        if not current_cells:
            break
            
        next_cells = []
        
        for curr_row, curr_col in current_cells:
            if (curr_row, curr_col) in visited:
                continue
            visited.add((curr_row, curr_col))
            
            # Check current cell elevation
            if elevation[curr_row, curr_col] <= 0:
                consecutive_ocean += 1
                if consecutive_ocean >= consecutive_limit:
                    return True  # Too many ocean cells, reject this outlet
            else:
                consecutive_ocean = 0  # Reset counter when we find land
            
            # Get upstream cells
            upstream = get_upstream_cells(curr_row, curr_col, fd)
            next_cells.extend(upstream)
        
        current_cells = next_cells
    
    return False

def is_tributary_outlet(row, col, area, fd, elevation, min_area, max_area, max_elev, 
                       consecutive_ocean_limit=3):
    """
    Check if a cell is a tributary outlet using two rules:
    
    Rule 1: Tributary joining larger river
    - Drainage area within range (min_area to max_area)
    - Elevation below threshold
    - Flows into a cell with drainage area > max_area (larger system)
    
    Rule 2: Isolated/coastal river outlet
    - Drainage area within range (min_area to max_area)
    - At outlet location (flows to smaller area, out of bounds, or elev ≤ 0 with upstream flow)
    - NO elevation threshold for coastal outlets
    - BUT reject if multiple consecutive ocean cells found upstream (indicates ocean location)
    """
    current_area = area[row, col]
    current_elev = elevation[row, col]
    
    # Check drainage area is in range
    if not (min_area <= current_area <= max_area):
        return False
    
    # Get downstream cell
    fd_value = fd[row, col]
    down_row, down_col = get_downstream_cell(row, col, fd_value)
    
    # Check if downstream cell is valid (within bounds)
    downstream_valid = (down_row is not None and down_col is not None and
                       0 <= down_row < area.shape[0] and 
                       0 <= down_col < area.shape[1])
    
    # Rule 2: Isolated/coastal river outlet conditions
    # Check if this is an outlet (flows out of bounds, to ocean, or at elevation ≤ 0)
    is_outlet = False
    
    # Condition B: Flows out of raster bounds
    if not downstream_valid:
        is_outlet = True
    
    # Condition C: Elevation at or below zero (sea level)
    # BUT only if drainage area is increasing upstream (real outlet, not ocean cell)
    # AND not too many consecutive ocean cells upstream
    if current_elev <= 0:
        # Check upstream cells to verify this is accumulating flow
        upstream_cells = get_upstream_cells(row, col, fd)
        has_upstream_flow = False
        
        for up_row, up_col in upstream_cells:
            upstream_area = area[up_row, up_col]
            # If upstream cell has smaller area, we're accumulating flow
            if upstream_area < current_area:
                has_upstream_flow = True
                break
        
        if has_upstream_flow:
            # Additional check: make sure we're not deep in the ocean
            # Reject if we find multiple consecutive ocean cells upstream
            if not check_upstream_path_for_ocean(row, col, elevation, fd, consecutive_ocean_limit):
                is_outlet = True
    
    # Condition A: Flows to cell with smaller or equal drainage area (outlet/ocean)
    if downstream_valid:
        downstream_area = area[down_row, down_col]
        if downstream_area <= current_area:
            is_outlet = True
    
    # If this is an outlet location, mark it (Rule 2)
    if is_outlet:
        return True
    
    # Rule 1: Tributary joining larger river
    # Must have valid downstream cell and meet elevation threshold
    if not downstream_valid:
        return False
    
    if current_elev > max_elev:
        return False
    
    downstream_area = area[down_row, down_col]
    
    # Tributary must be joining a stream larger than the max threshold
    if downstream_area > max_area:
        return True
    
    return False

# ============== MAIN PROCESSING ==============

print("Loading rasters...")
dem, dem_profile, dem_transform, dem_crs = load_raster(DEM_FILE)
area, area_profile, area_transform, area_crs = load_raster(AREA_FILE)
fd, fd_profile, fd_transform, fd_crs = load_raster(FD_FILE)

print(f"DEM shape: {dem.shape}")
print(f"Area range: {area.min():.2f} to {area.max():.2f} m²")
print(f"Elevation range: {dem.min():.2f} to {dem.max():.2f} m")
print(f"Using consecutive ocean limit: {CONSECUTIVE_OCEAN_LIMIT} cells")

# Find candidate cells
print("\nSearching for tributary outlets...")
outlets = []
count = 0

# Iterate through all cells
rows, cols = np.where(
    (area >= MIN_DRAINAGE_AREA) & 
    (area <= MAX_DRAINAGE_AREA) & 
    (dem <= MAX_ELEVATION) &
    (dem != dem_profile['nodata']) &
    (area != area_profile['nodata'])
)

print(f"Found {len(rows)} candidate cells within criteria...")

for idx in range(len(rows)):
    row, col = rows[idx], cols[idx]
    
    # Get UTM coordinates for spatial filtering
    x, y = xy(dem_transform, row, col)
    
    # Check if within X bounds
    if not (MIN_X <= x <= MAX_X):
        continue
    
    if is_tributary_outlet(row, col, area, fd, dem, 
                          MIN_DRAINAGE_AREA, MAX_DRAINAGE_AREA, MAX_ELEVATION,
                          CONSECUTIVE_OCEAN_LIMIT):
        
        outlets.append({
            'name': f'outlet_{count+1:04d}',
            'utm_x': x,
            'utm_y': y,
            'elevation_m': dem[row, col],
            'drainage_area_m2': area[row, col],
            'drainage_area_km2': area[row, col] / 1_000_000,
            'row': row,
            'col': col
        })
        
        count += 1
        
        if MAX_OUTLETS and count >= MAX_OUTLETS:
            print(f"Reached maximum of {MAX_OUTLETS} outlets (for testing)")
            break
    
    # Progress update
    if (idx + 1) % 10000 == 0:
        print(f"  Processed {idx+1}/{len(rows)} candidates... Found {count} outlets so far")

print(f"\nFound {len(outlets)} tributary outlets")

# Create DataFrame and save
df = pd.DataFrame(outlets)

if len(df) > 0:
    # Sort by drainage area
    df = df.sort_values('drainage_area_km2', ascending=False)
    
    # Save to CSV
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nSaved to: {OUTPUT_CSV}")
    
    # Print summary statistics
    print("\n=== SUMMARY ===")
    print(f"Total outlets: {len(df)}")
    print(f"Drainage area range: {df['drainage_area_km2'].min():.2f} - {df['drainage_area_km2'].max():.2f} km²")
    print(f"Elevation range: {df['elevation_m'].min():.2f} - {df['elevation_m'].max():.2f} m")
    print("\nFirst 10 outlets:")
    print(df[['name', 'utm_x', 'utm_y', 'elevation_m', 'drainage_area_km2']].head(10))
else:
    print("No outlets found matching criteria!")

Loading rasters...
DEM shape: (12203, 14877)
Area range: 748.05 to 27483837714.09 m²
Elevation range: -32768.00 to 6573.00 m
Using consecutive ocean limit: 2 cells

Searching for tributary outlets...
Found 499579 candidate cells within criteria...
  Processed 10000/499579 candidates... Found 13 outlets so far
  Processed 20000/499579 candidates... Found 35 outlets so far
  Processed 50000/499579 candidates... Found 63 outlets so far
  Processed 60000/499579 candidates... Found 66 outlets so far
  Processed 70000/499579 candidates... Found 67 outlets so far
  Processed 80000/499579 candidates... Found 73 outlets so far
  Processed 90000/499579 candidates... Found 83 outlets so far
  Processed 100000/499579 candidates... Found 94 outlets so far
  Processed 110000/499579 candidates... Found 101 outlets so far
  Processed 140000/499579 candidates... Found 127 outlets so far
  Processed 150000/499579 candidates... Found 145 outlets so far
  Processed 160000/499579 candidates... Found 150 ou